In [3]:
# Libraries
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, optimizers

# Helper Lib
import numpy as np

In [4]:
# Cifar is a set of 60K images 32 * 32 with 3 color channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)

# Constants
EPOCHS = 20
BATCH_SIZE = 128
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = optimizers.RMSprop()

In [3]:
# Build the model
def build(input_shape, classes):
    model = models.Sequential()
    
    # CONV => RELU => POOL with Dropout
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Flatten => RELU => Drop => Predict
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(classes, activation='softmax'))
    
    return model

In [4]:
# Load the dataset
((X_train, y_train), (X_test, y_test)) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step


In [5]:
# Normalize 
X_train, X_test = X_train / 255. , X_test / 255.

# Cast
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Conver class vectors to binary class vectors
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

In [6]:
model = build(INPUT_SHAPE, CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 27s 672us/sample - loss: 1.7559 - accuracy: 0.3809 - val_loss: 1.5403 - val_accuracy: 0.4271
Epoch 2/20
40000/40000 [==============================] - 25s 631us/sample - loss: 1.3947 - accuracy: 0.5106 - val_loss: 1.3613 - val_accuracy: 0.5169
Epoch 3/20
40000/40000 [==============================] - 26s 638us/sample - loss: 1.2619 - accuracy: 0.5543 - val_loss: 1.2124 - val_accuracy: 0.5799
Epoch 4/20
40000/40000 [==============================] - 27s 670us/sample - loss: 1.1674 - accuracy: 0.5922 - val_loss: 1.1665 - val_accuracy: 0.5991
Epoch 5/20
40000/40000 [==============================] - 85s 2ms/sample - loss: 1.0927 - accuracy: 0.6162 - val_loss: 1.1602 - val_accuracy: 0.5956
Epoch 6/20
40000/40000 [==============================] - 26s 640us/sample - loss: 1.0308 - accuracy: 0.6420 - val_loss: 1.0818 - val_accuracy: 0.6244
Epoch 7/20
40000/40000 [======================

In [8]:
# Evaluating the model
score = model.evaluate(X_test, y_test, verbose = VERBOSE)
print('\nTest Score: ', score[0])
print('Test Accuracy: ', score[1])

10000/10000 [==============================] - 25s 2ms/sample - loss: 0.9791 - accuracy: 0.6821

Test Score:  0.9791366603851318
Test Accuracy:  0.6821


##### Model 1 - One Layer CNN
Accuracy = 65.99%

## Improving the CIFAR-10 performance with a deeper network
- 1st Module: (CONV+CONV+MaxPool+DropOut)
- 2nd Module: (CONV+CONV+MaxPool+DropOut)
- 3rd Module: (CONV+CONV+MaxPool+DropOut)
- 4th Module: (Dense Layer)

In [1]:
def build_model():
    model = models.Sequential()
    
    # 1st Block
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))
    
    # 2nd Block 
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.3))

    # 3rd Block
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.4))

    # Dense
    model.add(layers.Flatten())
    model.add(layers.Dense(CLASSES, activation='softmax'))
    return model

In [2]:
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # Normalize
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    x_train = (x_train - mean) / (std + 1e-7)
    x_test = (x_test - mean) / (std + 1e-7)

    y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

    return x_train, y_train, x_test, y_test

In [ ]:
(x_train, y_train, x_test, y_test) = load_data()
deep_model = build_model()
deep_model.compile(loss='categorical_crossentropy',
                   optimizer='RMSprop',
                   metrics=['accuracy'])

# Train
batch_size = 64
deep_model.fit(x_train, y_train, batch_size=batch_size,
               epochs=EPOCHS, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 426s 9ms/sample - loss: 1.7342 - accuracy: 0.4803 - val_loss: 1.2540 - val_accuracy: 0.6138
Epoch 2/20
50000/50000 [==============================] - 441s 9ms/sample - loss: 1.0593 - accuracy: 0.6557 - val_loss: 1.0369 - val_accuracy: 0.6592
Epoch 3/20
50000/50000 [==============================] - 534s 11ms/sample - loss: 0.8544 - accuracy: 0.7126 - val_loss: 0.8221 - val_accuracy: 0.7305
Epoch 4/20
50000/50000 [==============================] - 400s 8ms/sample - loss: 0.7342 - accuracy: 0.7519 - val_loss: 0.7973 - val_accuracy: 0.7419
Epoch 5/20
50000/50000 [==============================] - 543s 11ms/sample - loss: 0.6523 - accuracy: 0.7772 - val_loss: 0.6474 - val_accuracy: 0.7837
Epoch 6/20
50000/50000 [==============================] - 431s 9ms/sample - loss: 0.5928 - accuracy: 0.7960 - val_loss: 0.6183 - val_accuracy: 0.7979
Epoch 7/20
50000/50000 [========================

In [6]:
score = deep_model.evaluate(x_test, y_test, batch_size=batch_size)
print('\nTest Score: ', score[0])
print('Test Accuracy: ', score[1])

10000/10000 [==============================] - 45s 4ms/sample - loss: 0.4761 - accuracy: 0.8459

Test Score:  0.47612907943725585
Test Accuracy:  0.8459


##### Model 2 - Deeper Layer CNN
Accuracy = 65.99%

## Model - 3 Improving the CIFAR-10 performance with data augmentation
We will use data augementation to use it in the deep model.